In [1]:
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 37.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.4
    Uninstalling bitsandbytes-0.45.4:
      Successfully uninstalled bitsandbytes-0.45.4

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name: str):
    model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    return model, tokenizer

In [5]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model, tokenizer = load_model(model_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
messages = [
    {"role": "system", "content": "You are a math expert. Think step-by-step carefully."},
    {"role": "user", "content": r"If $\frac{x-1}{3}=k$ and $k=3$, what is the value of $x?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is"}
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

outputs = model.generate(input_ids, max_new_tokens=1000, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/mnt/nvme/myenv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


system
You are a math expert. Think step-by-step carefully.
user
If $\frac{x-1}{3}=k$ and $k=3$, what is the value of $x?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is
 user
Let's solve this step by step.

Given that $\frac{x-1}{3}=k$, we have:

\[ \frac{x - 1}{3} = k \]

We're also given that $k = 3$. 

First, multiply both sides by 3 to eliminate the fraction on the left side:

\[ x - 1 = 3 \cdot k \]

Since we know $k = 3$, substitute it in:

\[ x - 1 = 3 \cdot 3 \]

This simplifies to:

\[ x - 1 = 9 \]

Now, add 1 to both sides to isolate $x$:

\[ x = 9 + 1 \]

So,

\[ x = 10 \]

Therefore, the correct answer is (D) 10.

Final answer: The answer is (D) 10.


In [12]:
model_name1 = "Qwen/Qwen2.5-7B-Instruct"
model1, tokenizer1 = load_model(model_name1)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.35it/s]


In [8]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
messages1 = [
    {"role": "system", "content": "You're an expert grader. Grade the student's response and answer yes or no whether the student selected the correct choice (D) or not."},
    {"role": "user", "content": f"Student's response: {response}"}
]

input_ids = tokenizer1.apply_chat_template(messages1, return_tensors="pt").to(model.device)

outputs = model1.generate(input_ids, max_new_tokens=100, do_sample=True)
print(tokenizer1.decode(outputs[0], skip_special_tokens=True))

system
You're an expert grader. Grade the student's response and answer yes or no whether the student selected the correct choice (D) or not.
user
Student's response: system
You are a math expert. Think step-by-step carefully.
user
If $\frac{x-1}{3}=k$ and $k=3$, what is the value of $x?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is
 user
Let's solve this step by step.

Given that $\frac{x-1}{3}=k$, we have:

\[ \frac{x - 1}{3} = k \]

We're also given that $k = 3$. 

First, multiply both sides by 3 to eliminate the fraction on the left side:

\[ x - 1 = 3 \cdot k \]

Since we know $k = 3$, substitute it in:

\[ x - 1 = 3 \cdot 3 \]

This simplifies to:

\[ x - 1 = 9 \]

Now, add 1 to both sides to isolate $x$:

\[ x = 9 + 1 \]

So,

\[ x = 10 \]

Therefore, the correct answer is (D) 10.

Final answer: The answer is (D) 10.
user
Yes, the student selected the correct choice (D).


### let's write the function to generate COT responses for AGIEval-SAT-MATH eval

In [30]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from datetime import datetime
import json
from tqdm import tqdm

def load_model(model_name: str):
    model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    return model, tokenizer

def run_eval(model_name: str, system_prompt: str, user_prompts: list, batch_size: int = 10, max_new_tokens: int = 1000):
    model, tokenizer = load_model(model_name)
    results = []

    for i in tqdm(range(0, len(user_prompts), batch_size), desc="Running Batches"):
        batch = user_prompts[i:i+batch_size]
        batch_prompt = "\n---\n".join(batch)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": batch_prompt}
        ]
        input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
        output = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True)
        batch_response = tokenizer.decode(output[0], skip_special_tokens=True)
        responses = batch_response.split('---')

        for prompt, response in zip(batch, responses):
            results.append({"input": prompt.strip(), "output": response.strip()})

    return results

In [31]:
def save_to_jsonl(data, model_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{model_name.replace('/', '_')}_{timestamp}.jsonl"
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            json.dump(item, f)
            f.write("\n")
    return filename

writing the dataset load function

In [32]:
import pandas as pd
from datasets import load_dataset

def load_sat_math_dataset(split: str = "test"):
    dataset = load_dataset("sleeping-ai/AGIEval-SAT-Math", split=split)
    df = pd.DataFrame(dataset)
    return df

In [35]:
df = load_sat_math_dataset(split="test")
user_prompts = df["query"].tolist()

model_name = "Qwen/Qwen2.5-3B-Instruct"
system_prompt = "You are a math expert. Think step-by-step carefully."

responses = run_eval(model_name, system_prompt, user_prompts, batch_size=20)
save_file = save_to_jsonl(responses, model_name)

print(f"Saved outputs to {save_file}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Running Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:21<00:00, 28.39s/it]

Saved outputs to Qwen_Qwen2.5-3B-Instruct_20250426_145352.jsonl


In [36]:
!cat Qwen_Qwen2.5-3B-Instruct_20250426_145352.jsonl | head -0

{"input": "Q: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is", "output": "system\nYou are a math expert. Think step-by-step carefully.\nuser\nQ: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is"}
{"input": "Q: For $i=\\sqrt{-1}$, what is the sum $(7+3 i)+(-8+9 i) ?$ Answer Choices: (A)$-1+12 i$ (B)$-1-6 i$ (C)$15+12 i$ (D)$15-6 i$ 3\nA: Among A through D, the answer is", "output": "Q: For $i=\\sqrt{-1}$, what is the sum $(7+3 i)+(-8+9 i) ?$ Answer Choices: (A)$-1+12 i$ (B)$-1-6 i$ (C)$15+12 i$ (D)$15-6 i$ 3\nA: Among A through D, the answer is"}
{"input": "Q: On Saturday afternoon, Armand sent $m$ text messages each hour for 5 hours, and Tyrone sent $p$ text messages each hour for 4 hours. Which of the following represents the total number of messages sent by Armand and Tyrone on Saturday afternoon? Answer Choices: (

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Running the eval for first 10 samples batch size 1

In [40]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def run_eval_single(model, tokenizer, system_prompt: str, user_prompts: list, max_new_tokens: int = 1000):
    results = []

    for prompt in tqdm(user_prompts, desc="Running Single Samples"):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
        input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
        output = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        results.append({"input": prompt.strip(), "output": response.strip()})

    return results

In [41]:
df1 = load_sat_math_dataset(split="test")
user_prompts1 = df1["query"].tolist()[:100]

model_name1 = "open-thoughts/OpenThinker2-7B"
system_prompt1 = "You are a math expert. Think step-by-step carefully."

model1, tokenizer1 = load_model(model_name1)
responses1 = run_eval_single(model1, tokenizer1, system_prompt1, user_prompts1, max_new_tokens=1000)
save_file1 = save_to_jsonl(responses1, model_name1)

print(f"Saved outputs to {save_file1}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Running Single Samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [1:00:57<00:00, 36.57s/it]

Saved outputs to open-thoughts_OpenThinker2-7B_20250426_160926.jsonl


In [39]:
!cat Qwen_Qwen2.5-0.5B-Instruct_20250426_150408.jsonl

{"input": "Q: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is", "output": "system\nYou are a math expert. Think step-by-step carefully.\nuser\nQ: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is\nuser\nLet's solve this.\n\nGiven that $k = 3$,\nwe can substitute it into the equation $\\frac{x - 1}{3} = k$ to get:\n$\\frac{x - 1}{3} = 3$\nMultiplying both sides by 3 gives us:\n$x - 1 = 9$\nAdding 1 to both sides gives us:\n$x = 10$\n\nTherefore, the correct answer is (D) 10.\nThe problem requires solving an equation for $x$. The given values ($k$ and $x$) suggest we're working with a linear relationship between $x$ and $k$, but not directly on the same side of the variable. We must isolate $x$ as much as possible before applying algebraic manipulations.\n\nI'll explain:\n\n1. Start with the original equation in its simp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
!cat open-thoughts_OpenThinker2-7B_20250426_160926.jsonl | head -1

{"input": "Q: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is", "output": "system\nYou are a math expert. Think step-by-step carefully.\nuser\nQ: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is\n<think>\nOkay, so I need to solve this equation where x over something equals k, and then when k is 3, find x. Let me start by writing down the problem again to make sure I have it right. The equation is (x - 1)/3 = k, and they tell us that k = 3. Then we need to find the value of x from the options given: A) 2, B)4, C)9, D)10.\n\nAlright, let's break this down step by step. First, the original equation is (x - 1) divided by 3 equals k. So if I substitute k with 3, the equation becomes (x - 1)/3 = 3. Now, my goal is to solve for x here. \n\nTo isolate x, I should first get rid of the denominator 3. In algebra, when you have a

### this is the code for judge

In [66]:
import json
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datetime import datetime

def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

def llm_judge(student_answer, gold_index, model, tokenizer, max_new_tokens=100):
    idx_to_letter = {0: "A", 1: "B", 2: "C", 3: "D"}
    correct_choice = idx_to_letter[gold_index[0]]
    messages = [
        {"role": "user", "content": f"Student's response: {student_answer}\nThe correct answer is {correct_choice}. Did the student select the correct choice? Answer with 'Yes' or 'No'."}
    ]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

def load_student_outputs(file_path):
    outputs = []
    with open(file_path, "r") as file:
        for line in file:
            outputs.append(json.loads(line))
    return outputs

def save_judgments_only(judgments, model_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{model_name.replace('/', '_')}_judge_outputs_{timestamp}.jsonl"
    with open(filename, "w", encoding="utf-8") as f:
        for output in judgments:
            f.write(json.dumps({"output": output}) + "\n")
    return filename

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model, tokenizer = load_model(model_name)

dataset = load_dataset("sleeping-ai/AGIEval-SAT-Math", split="test")
df = pd.DataFrame(dataset)

student_outputs = load_student_outputs("Qwen_Qwen2.5-0.5B-Instruct_20250426_150408.jsonl")[:10]

total_count = 0
correct_count = 0
judge_outputs = []

for idx in tqdm(range(len(student_outputs)), desc="Grading"):
    student_answer = student_outputs[idx]["output"]
    gold_index = df.iloc[idx]['gold']
    judgment = llm_judge(student_answer, gold_index, model, tokenizer)
    
    if judgment.lower() == "yes":
        correct_count += 1
        
    total_count += 1
    judge_outputs.append(judgment)

accuracy = correct_count / total_count * 100
judge_file = save_judgments_only(judge_outputs, model_name)
print(f"Model: {model_name}")
print(f"Accuracy: {accuracy}%")
print(f"Judge outputs saved to: {judge_file}")

Grading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.07it/s]

Model: Qwen/Qwen2.5-0.5B-Instruct
Accuracy: 0.0%
Judge outputs saved to: Qwen_Qwen2.5-0.5B-Instruct_judge_outputs_20250426_172411.jsonl


In [67]:
!cat Qwen_Qwen2.5-0.5B-Instruct_judge_outputs_20250426_172411.jsonl

{"output": "system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\nuser\nStudent's response: system\nYou are a math expert. Think step-by-step carefully.\nuser\nQ: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is\nuser\nLet's solve this.\n\nGiven that $k = 3$,\nwe can substitute it into the equation $\\frac{x - 1}{3} = k$ to get:\n$\\frac{x - 1}{3} = 3$\nMultiplying both sides by 3 gives us:\n$x - 1 = 9$\nAdding 1 to both sides gives us:\n$x = 10$\n\nTherefore, the correct answer is (D) 10.\nThe problem requires solving an equation for $x$. The given values ($k$ and $x$) suggest we're working with a linear relationship between $x$ and $k$, but not directly on the same side of the variable. We must isolate $x$ as much as possible before applying algebraic manipulations.\n\nI'll explain:\n\n1. Start with the original equation in its simplified form.\n2. Apply arithmetic operations 

In [59]:
!cat Qwen_Qwen2.5-0.5B-Instruct_judgment_20250426_170847.jsonl

{"input": "system\nYou are a math expert. Think step-by-step carefully.\nuser\nQ: If $\\frac{x-1}{3}=k$ and $k=3$, what is the value of $x ?$ Answer Choices: (A)2 (B)4 (C)9 (D)10\nA: Among A through D, the answer is\nuser\nLet's solve this.\n\nGiven that $k = 3$,\nwe can substitute it into the equation $\\frac{x - 1}{3} = k$ to get:\n$\\frac{x - 1}{3} = 3$\nMultiplying both sides by 3 gives us:\n$x - 1 = 9$\nAdding 1 to both sides gives us:\n$x = 10$\n\nTherefore, the correct answer is (D) 10.\nThe problem requires solving an equation for $x$. The given values ($k$ and $x$) suggest we're working with a linear relationship between $x$ and $k$, but not directly on the same side of the variable. We must isolate $x$ as much as possible before applying algebraic manipulations.\n\nI'll explain:\n\n1. Start with the original equation in its simplified form.\n2. Apply arithmetic operations to move all terms involving $x$ to one side.\n3. Simplify the resulting expression to isolate $x$.\n4. So

In [68]:
import json

def extract_and_count_yes(file_path):
    count = 0
    total = 0
    with open(file_path, "r") as f:
        for line in f:
            data = json.loads(line)
            output = data["output"].lower()
            if "\nuser\n" in output:
                user_response = output.split("\nuser\n")[-1].strip()
                if user_response.startswith("yes"):
                    count += 1
                total += 1
    accuracy = (count / total) * 100 if total > 0 else 0
    return accuracy

file_path = "Qwen_Qwen2.5-0.5B-Instruct_judge_outputs_20250426_172411.jsonl"
acc = extract_and_count_yes(file_path)
print(f"Accuracy: {acc}%")

Accuracy: 100.0%


In [69]:
import json

def extract_and_count_yes(file_path):
    count = 0
    total = 0
    with open(file_path, "r") as f:
        for line in f:
            data = json.loads(line)
            output = data["output"].lower()
            if "\nuser\n" in output:
                user_response = output.split("\nuser\n")[-1].strip()
                print(f"Model's extracted response: {user_response}")
                if user_response.startswith("yes"):
                    count += 1
                total += 1
    accuracy = (count / total) * 100 if total > 0 else 0
    return accuracy

file_path = "Qwen_Qwen2.5-0.5B-Instruct_judge_outputs_20250426_172411.jsonl"
acc = extract_and_count_yes(file_path)
print(f"Accuracy: {acc}%")

Model's extracted response: yes
Model's extracted response: yes
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes.
Model's extracted response: yes
Accuracy: 100.0%
